# Installation

pip install -U keras-tuner

### Optional (Only if errors appear)

1. pip install tensorflow --upgrade --user
2. pip install keras --upgrade --user

In [1]:
import pandas as pd
import numpy as np

dataset=pd.read_csv('cardio_dataset.csv')
print(dataset.head())
dataset=dataset.values

   SEX  AGEIR   TC  HDL  SMOKE_  BPMED  DIAB_01  RISK
0    1     48  236   66       0      1        0   1.1
1    0     48  260   51       0      1        1   7.0
2    0     44  187   49       1      1        0   7.0
3    1     42  216   57       1      1        0   0.4
4    1     56  156   42       0      1        0   2.2


In [2]:
data=dataset[:,0:7]
target=dataset[:,7]

In [3]:
from sklearn.preprocessing import MinMaxScaler

target=np.reshape(target, (-1,1))

scaler_data = MinMaxScaler(feature_range=(0,1))
scaler_target = MinMaxScaler()

data_scaled=scaler_data.fit_transform(data)
target_scaled=scaler_target.fit_transform(target)

In [4]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled,test_size=0.2)

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy as np

from keras.optimizers import Adam,Adagrad,Adadelta

def build_model(parameters):
    
    model = Sequential()
    
    for i in range(parameters.Int('num_layers', 2, 20)):

        if(i==0):
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                            activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh']),input_dim=7))
            
            model.add(Dropout(parameters.Choice('drop_prob '+str(i),[0.2,0.3,0.4,0.5])))
        
        else:
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                                   activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh'])))
            
            model.add(Dropout(parameters.Choice('drop_prob '+str(i),[0.2,0.3,0.4,0.5])))
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer=parameters.Choice('optmz', ['adam', 'adaDelta', 'adaGrad']),loss=parameters.Choice('loss f', ['mse', 'mae']))
    #model.compile(optimizer=Adam(parameters.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),loss='mse')
    
    return model

In [7]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,objective='val_loss',max_trials=5,executions_per_trial=3,directory='project',
                     project_name='heart-risk')

INFO:tensorflow:Reloading Oracle from existing project project\heart-risk\oracle.json


[more info](https://keras-team.github.io/keras-tuner/)

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 10
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
#neurons layer0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 1 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
#neurons layer1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 2 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
optmz (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adaDelta', 'adaGrad'], 'ordered': False}
loss f (Choi

In [9]:
tuner.search(train_data,train_target,epochs=10,validation_data=(test_data, test_target))

Trial 5 Complete [00h 01m 09s]
val_loss: 0.022785576681296032

Best val_loss So Far: 0.0051856329664587975
Total elapsed time: 00h 03m 02s
INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

Results summary
Results in project\heart-risk
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 4
#neurons layer0: 352
activation_function 0: tanh
drop_prob 1: 0.2
#neurons layer1: 480
activation_function 1: sigmoid
drop_prob 2: 0.3
optmz: adam
loss f: mse
drop_prob 0: 0.2
#neurons layer2: 288
activation_function 2: relu
#neurons layer3: 448
activation_function 3: tanh
drop_prob 3: 0.4
#neurons layer4: 480
activation_function 4: tanh
drop_prob 4: 0.4
#neurons layer5: 96
activation_function 5: sigmoid
drop_prob 5: 0.4
#neurons layer6: 320
activation_function 6: relu
drop_prob 6: 0.4
#neurons layer7: 128
activation_function 7: relu
drop_prob 7: 0.2
#neurons layer8: 160
activation_function 8: relu
drop_prob 8: 0.2
#neurons layer9: 416
activation_function 9: tanh
drop_prob 9: 0.2
#neurons layer10: 64
activation_function 10: tanh
drop_prob 10: 0.4
#neurons layer11: 480
activation_function 11: relu
drop_prob 11: 0.5
#neurons layer12

In [11]:
best_models = tuner.get_best_models()
print(best_models[0].summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 352)               2816      
_________________________________________________________________
dropout (Dropout)            (None, 352)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 480)               169440    
_________________________________________________________________
dropout_1 (Dropout)          (None, 480)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 288)               138528    
_________________________________________________________________
dropout_2 (Dropout)          (None, 288)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 448)               1